# OC-IA-P10 DATA EXPLORATION

In [ ]:
# import sys
# !{sys.executable} -m pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
frames = pd.read_json('frames/frames.json')

In [4]:
frames = frames[['turns', 'labels']]

In [5]:
frames

,turns,labels
0,[{'text': 'I'd like to book a trip to Atlantis...,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,"[{'text': 'Hello, I am looking to book a vacat...","{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,[{'text': 'Hello there i am looking to go on a...,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,[{'text': 'Hi I'd like to go to Caprica from B...,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,"[{'text': 'Hello, I am looking to book a trip ...","{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
...,...,...
1364,[{'text': 'Hi I've got 9 days free and I'm loo...,"{'userSurveyRating': 3.5, 'wizardSurveyTaskSuc..."
1365,[{'text': 'I need to get to Fortaleza on Septe...,"{'userSurveyRating': 4.5, 'wizardSurveyTaskSuc..."
1366,[{'text': 'We're finally going on vacation isn...,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
1367,"[{'text': 'Hi there, I'm looking for a place t...","{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."


In [6]:
data = pd.DataFrame()
for i in tqdm(frames.index):
    data = pd.concat((data, pd.DataFrame(frames.loc[i]['turns'])), ignore_index=True)

data = data[data['author']=='user']
data = data[['text', 'labels']] # drop author, timestamp, db
data = data.reset_index(drop=True)

  0%|          | 0/1369 [00:00<?, ?it/s]

What are the different kind of interactions?

In [7]:
set([item['name'] for i in data.index for item in data['labels'][i]['acts_without_refs']])

{'affirm',
 'confirm',
 'goodbye',
 'greeting',
 'inform',
 'moreinfo',
 'negate',
 'request',
 'request_alts',
 'request_compare',
 'switch_frame',
 'thankyou'}

In [8]:
data['acts_without_refs'] = data['labels'].apply(lambda x: x['acts_without_refs'])

In [9]:
data

,text,labels,acts_without_refs
0,I'd like to book a trip to Atlantis from Capri...,"{'acts': [{'args': [{'val': 'book', 'key': 'in...","[{'args': [{'val': 'book', 'key': 'intent'}], ..."
1,"Yes, how about going to Neverland from Caprica...","{'acts': [{'args': [{'val': 'Neverland', 'key'...","[{'args': [{'val': 'Neverland', 'key': 'dst_ci..."
2,I have no flexibility for dates... but I can l...,"{'acts': [{'args': [{'val': False, 'key': 'fle...","[{'args': [{'val': False, 'key': 'flex'}], 'na..."
3,I suppose I'll speak with my husband to see if...,"{'acts': [{'args': [], 'name': 'thankyou'}], '...","[{'args': [], 'name': 'thankyou'}]"
4,"Hello, I am looking to book a vacation from Go...","{'acts': [{'args': [{'val': 'book', 'key': 'in...","[{'args': [{'val': 'book', 'key': 'intent'}], ..."
...,...,...,...
10402,"5 adults and 7 kids! Yup, the lot of us. We wa...","{'acts': [{'args': [{'val': '5', 'key': 'n_adu...","[{'args': [{'val': '5', 'key': 'n_adults'}, {'..."
10403,Oh yes! Between September 12 and 26!,"{'acts': [{'args': [{'val': 'September 12', 'k...","[{'args': [{'val': 'September 12', 'key': 'str..."
10404,"That sounds amazing, and it's within those dat...",{'acts': [{'args': [{'val': [{'annotations': [...,"[{'args': [{'val': None, 'key': 'str_date'}, {..."
10405,"Ok perfect, book me!","{'acts': [{'args': [{'val': 'book', 'key': 'in...","[{'args': [{'val': 'book', 'key': 'intent'}], ..."


In [10]:
def extract_values(list_of_values):
    """
    Return dict {key1:value1, key2:value2} from list shaped
    as [[{'val':value1, 'key':key1}], [{'val':value2, 'key':key2}]].
    Inner lists can contain several dicts.
    """
    if not list_of_values:
        return None
    result = []
    required_keys = [
        'dst_city', 
        'or_city', 
        'str_date', 
        'end_date',
        'budget', 
        'max_duration'
        ]
    for item in list_of_values:
        for dict_of_elements in item:
            result.append((dict_of_elements['key'], dict_of_elements['val'])) if dict_of_elements['key'] in required_keys else None       
    return result or None

In [11]:
data['values'] = data['acts_without_refs'].apply(lambda x: [item['args'] for item in x if item['name']=='inform'])
data['values'] = data['values'].apply(extract_values)

In [12]:
data['moreinfo'] = data['acts_without_refs'].apply(lambda x: [item['args'] for item in x if item['name']=='moreinfo'])
data['moreinfo'] = data['moreinfo'].apply(extract_values)

In [13]:
data[data['moreinfo'].notna()]

,text,labels,acts_without_refs,values,moreinfo
5588,Can you tell me a bit more about the Ciudad Ju...,{'acts': [{'args': [{'val': [{'annotations': [...,"[{'args': [{'val': 'Ciudad Juarez', 'key': 'ds...",None,"[(dst_city, Ciudad Juarez)]"


The column `moreinfo` is useless and we can delete it.

In [14]:
data.drop(columns=['moreinfo', 'labels', 'acts_without_refs'], inplace=True, errors='ignore')

In [15]:
data = data.dropna(subset=['values'])
data.reset_index(drop=True, inplace=True)

In [17]:
data

,text,values
0,I'd like to book a trip to Atlantis from Capri...,"[(dst_city, Atlantis), (or_city, Caprica), (st..."
1,"Yes, how about going to Neverland from Caprica...","[(dst_city, Neverland), (budget, 1900), (or_ci..."
2,I have no flexibility for dates... but I can l...,"[(or_city, Atlantis)]"
3,"Hello, I am looking to book a vacation from Go...","[(dst_city, Mos Eisley), (or_city, Gotham City..."
4,What about a trip from Gotham City to Neverlan...,"[(dst_city, Neverland), (or_city, Gotham City)..."
...,...,...
4956,"Fine, if you're going to be like that... I gue...","[(or_city, Frankfurt)]"
4957,Is 3.5 stars the best you can do? Are you seri...,"[(dst_city, St. Louis)]"
4958,I need to book a trip for the whole family fro...,"[(or_city, Tampa), (dst_city, -1)]"
4959,"5 adults and 7 kids! Yup, the lot of us. We wa...","[(budget, 32800)]"


In [19]:
data.loc[4958]['text']

"I need to book a trip for the whole family from Tampa but we don't know where to go. Can you help us?"